In [ ]:
import tkinter as tk
from tkinter import filedialog, scrolledtext
import pickle
import re
from PyPDF2 import PdfReader

# Load the trained model and vectorizer
clf = pickle.load(open('clf.pkl', 'rb'))
tfidfd = pickle.load(open('tfidf.pkl', 'rb'))

def clean_resume(resume_text):
    clean_text = re.sub('http\S+\s*', ' ', resume_text)
    clean_text = re.sub('RT|cc', ' ', clean_text)
    clean_text = re.sub('#\S+', '', clean_text)
    clean_text = re.sub('@\S+', '  ', clean_text)
    clean_text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', clean_text)
    clean_text = re.sub(r'[^\x00-\x7f]', r' ', clean_text)
    clean_text = re.sub('\s+', ' ', clean_text)
    return clean_text

def analyze_resume():
    # Check if a file has been uploaded
    if not file_path.get():
        result_text.delete('1.0', tk.END)
        result_text.insert(tk.END, "Please upload a resume file first.\n")
        return
    
    # Try to read the uploaded file
    try:
        file_ext = file_path.get().split('.')[-1]
        if file_ext == 'txt':
            with open(file_path.get(), 'r', encoding='utf-8') as file:
                resume_text = file.read()
        elif file_ext == 'pdf':
            resume_text = ''
            with open(file_path.get(), 'rb') as file:
                reader = PdfReader(file)
                for page in reader.pages:
                    resume_text += page.extract_text()
        else:
            result_text.delete('1.0', tk.END)
            result_text.insert(tk.END, "Unsupported file format. Please upload a .txt or .pdf file.\n")
            return
    except Exception as e:
        result_text.delete('1.0', tk.END)
        result_text.insert(tk.END, f"Error reading file: {e}\n")
        return

    # Clean the resume text
    cleaned_resume = clean_resume(resume_text)
    
    # Transform the cleaned resume using the trained TfidfVectorizer
    input_features = tfidfd.transform([cleaned_resume])
    
    # Make the prediction using the loaded classifier
    prediction_id = clf.predict(input_features)[0]
    
    # Map category ID to category name
    category_mapping = {
        15: "Java Developer",
        23: "Testing",
        8: "DevOps Engineer",
        20: "Python Developer",
        24: "Web Designing",
        12: "HR",
        13: "Hadoop",
        3: "Blockchain",
        10: "ETL Developer",
        18: "Operations Manager",
        6: "Data Science",
        22: "Sales",
        16: "Mechanical Engineer",
        1: "Arts",
        7: "Database",
        11: "Electrical Engineering",
        14: "Health and fitness",
        19: "PMO",
        4: "Business Analyst",
        9: "DotNet Developer",
        2: "Automation Testing",
        17: "Network Security Engineer",
        21: "SAP Developer",
        5: "Civil Engineer",
        0: "Advocate",
    }
    
    # Get the category name from the mapping
    category_name = category_mapping.get(prediction_id, "Unknown")
    
    # Display the predicted category
    result_text.delete('1.0', tk.END)
    result_text.insert(tk.END, f"Predicted Category: {category_name}\n")

# Create the Tkinter window
root = tk.Tk()
root.title("Resume Screening App")

# Create widgets
file_path = tk.StringVar()

upload_button = tk.Button(root, text="Upload Resume", command=lambda: file_path.set(filedialog.askopenfilename(filetypes=[("Text files", "*.txt"), ("PDF files", "*.pdf")])))
upload_button.grid(row=0, column=0, padx=10, pady=10)

result_button = tk.Button(root, text="Analyze Resume", command=analyze_resume)
result_button.grid(row=0, column=1, padx=10, pady=10)

result_text = scrolledtext.ScrolledText(root, width=60, height=15)
result_text.grid(row=1, column=0, columnspan=2, padx=10, pady=10)

# Start the Tkinter event loop
root.mainloop()
